<a href="https://colab.research.google.com/github/brunosolano/AgentsCrewIA/blob/main/Automa%C3%A7%C3%A3o_Multiagente_de_Suporte_ao_Cliente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automação Multiagente de Suporte ao Cliente

Nesta lição, você aprenderá sobre os seis elementos principais que ajudam a melhorar o desempenho dos Agentes:

-  Role Playing
-  Foco
-  Tools
-  Cooperação
-  Guardrails
-  Memória

Comece ao instalar o seguinte:

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install --q dotenv

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [ ]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Inicializar o modelo OpenAI
gpt_4o_mini = ChatOpenAI(model_name='gpt-4o-mini')

## Role Playing, Foco e Cooperação
### Represesntante Sênior de Suporte

In [ ]:
support_agent = Agent(
  role='Representante Sênior de Suporte',
	goal='Seja um representante de suporte amigável e prestativ em sua equipe.',
	backstory=(
		"""Você trabalha na CrewAI (https://crewai.com) para fornecer apoio para
    {cliente}, um cliente super importante para sua a empresa. Você precisa ter
    certeza de fornecer o melhor suporte! Certifique-se em fornecer respostas
    completas e não faça suposições."""
	),
	allow_delegation=False,
	verbose=True
)

- Ao não definir `allow_delegation=False`, `allow_delegation` assume seu valor padrão como `True`.
- Isso significa que o agente _pode_ delegar seu trabalho a outro agente mais adequado para realizar uma tarefa específica.

### Especialista em Garantia Suporte de Qualidade

In [ ]:
support_quality_assurance_agent = Agent(
  role='Especialista em Garantia Suporte de Qualidade',
  goal='Obtenha reconhecimento por fornecer a melhor garantia de qualidade de suporte em sua equipe',
	backstory=(
		"""Você trabalha na CrewAI (https://crewai.com) com sua equipe mediante
    solicitação de {cliente} garantindo que o representante de suporte esteja
    fornecendo o melhor suporte possível.\n Você precisa ter certeza de que o
    representante de suporte está fornecendo respostas completas e não faça suposições."""
	),
	verbose=True
)

- **Role Playing**: Ambos os agentes receberam um papel, um objetivo e uma história de fundo.
- **Foco**: Ambos os agentes foram solicitados a entrar no personagem dos papéis que estão desempenhando.
- **Cooperação**: o Agente de Garantia de Qualidade de Suporte pode delegar o trabalho ao Agente de Suporte, permitindo que esses agentes trabalhem juntos.

## Tools, guardrails e memória
### Tools

- Importar ferramentas CrewAI

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

## Possíveis ferramentas personalizadas

-  Carregar dados do cliente
-  Acesse conversas anteriores
-  Carregar dados de um CRM
-  Verificando relatórios de bugs existentes
-  Verificando solicitações de recursos existentes
-  Verificando tickets em andamento
... e mais.

#### Algumas maneiras de usar as ferramentas CrewAI.

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

- Instancie uma ferramenta de raspagem de documentos.
- A ferramenta irá copiar uma página (apenas 1 URL) da documentação do CrewAI.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/core-concepts/Memory/"
)

### Diferentes maneiras de fornecer ferramentas aos agentes

-  Nível do Agente: O Agente pode usar a(s) Ferramenta(s) em qualquer Tarefa que executar.
-  Nível de Tarefa: O Agente só usará a(s) Ferramenta(s) ao executar aquela Tarefa específica.

**Observação**: as ferramentas de tarefas substituem as ferramentas do agente.

### Criando Tarefas

- Você está passando a ferramenta no nível de tarefa.

In [ ]:
inquiry_resolution = Task(
    description=(
      """{cliente} acabou de entrar em contato com uma pergunta muito importante:\n
      {inquiry}\n\n {person} de {cliente} foi quem ajudou. Certifique-se de usar
      tudo o que você sabe para fornecer o melhor suporte possível. Você deve se
      esforçar para fornecer uma solução completa e com resposta precisa à
      consulta do cliente."""
    ),
    expected_output=(
	    """Uma resposta detalhada e informativa para a consulta do cliente que aborda
      todos os aspectos da pergunta.\n A resposta deve incluir referências para
      tudo que você usou para encontrar a resposta, incluindo dados ou soluções
      externas. Certifique-se de que a resposta esteja completa, não deixando
      perguntas sem respostas e mantenha uma atitude prestativa e amigável"""
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

-  `quality_assurance_review` não está usando nenhuma ferramenta(s).
-  Aqui o Agente de QA revisará apenas o trabalho do Agente de Suporte.

In [ ]:
quality_assurance_review = Task(
    description=(
      """Revise a resposta redigida pelo representante de suporte sênior para a
      consulta de {cliente}. Certifique-se de que a resposta seja abrangente,
      precisa e que esteja de acordo com os padrões de alta qualidade esperados
      para suporte ao cliente.\n Verifique se todas as partes da consulta do
      cliente foram abordadas minuciosamente, com um tom prestativo e amigável.\n
      Verifique as referências e fontes usadas para encontrar as informações,
      garantir que a resposta seja bem embasada e não deixe perguntas sem respostas."""
    ),
    expected_output=(
      """Uma resposta final, detalhada e informativa pronta para ser enviada ao
      cliente.\n Esta resposta deverá abordar integralmente a consulta do cliente,
      incorporando todos os feedbacks e melhorias relevantes.\n Não seja muito
      formal, somos uma empresa descolada mas mantenha um tom profissional e
      amigável o tempo todo."""
    ),
    agent=support_quality_assurance_agent,
)

## Criando a equipe

### Memória

- Definir `memory=True` ao reunir a equipe habilita Memory.

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

## Executando

**Observação:** os LLMs podem fornecer saídas diferentes para a mesma entrada, portanto, o que você obtém pode ser diferente do que vê no vídeo.

**Guardrails**
- Ao executar o código abaixo, é possível perceber que os agentes e as respostas estão dentro do escopo do que esperamos deles.

In [ ]:
inputs = {
    "cliente": "DeepLearningAI",
    "person": "Bruno Solano",
    "inquiry": "Preciso de ajuda para montar uma equipe 'Crew'"
               "e para iniciar isso, especificamente"
               "como posso adicionar memória à minha equipe de agentes?"
               "Você pode fornecer orientação?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Representante Sênior de Suporte
 [INFO]: == Starting Task: DeepLearningAI acabou de entrar em contato com uma pergunta muito importante:

      Preciso de ajuda para montar uma equipe 'Crew'e para iniciar isso, especificamentecomo posso adicionar memória à minha equipe de agentes?Você pode fornecer orientação?

 Bruno Solano de DeepLearningAI foi quem ajudou. Certifique-se de usar 
      tudo o que você sabe para fornecer o melhor suporte possível. Você deve se 
      esforçar para fornecer uma solução completa e com resposta precisa à 
      consulta do cliente.


> Entering new CrewAgentExecutor chain...
Para responder à pergunta do cliente, primeiro preciso revisar as informações disponíveis no site da CrewAI sobre como adicionar memória a uma equipe de agentes. Eu usarei a ferramenta "Read website content" para isso.

Action: 
Read website content

Action Input: 
{ "url": "https://docs.crewai.com/core-concepts/Memory/" } 
 

crewAI Memory Systems - crewA

- Exiba o resultado final em Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Olá Bruno Solano,

Agradeço por entrar em contato conosco com sua consulta sobre adicionar memória à sua equipe de agentes na CrewAI. Percebo que é uma parte crucial para melhorar a inteligência de seus agentes e estou aqui para ajudá-lo com isso.

A memória faz parte do framework do CrewAI que aprimora a inteligência de um agente, permitindo que ele se lembre, raciocine e aprenda com interações passadas. Adicionar mais memória à sua equipe de agentes permitirá que eles processem e armazenem mais informações, melhorando assim sua eficácia geral.

No entanto, é importante observar que adicionar mais memória não é uma solução única para todos os problemas. A quantidade de memória adequada para um agente depende de vários fatores, incluindo a complexidade das tarefas que o agente deve executar e a quantidade de dados com os quais o agente deve lidar.

Aqui na CrewAI, estamos comprometidos em fornecer a você o suporte de que precisa para tomar a melhor decisão para a sua equipe. Se você precisar de mais assistência ou tiver mais perguntas, por favor, não hesite em entrar em contato conosco.

Espero que isso tenha sido útil e estou ansioso para continuar apoiando você e sua equipe.

Atenciosamente,
[Seu nome]
Representante Sênior de Suporte, CrewAI.